# Train Models using keras

## Dataset

If you don't have the dataset ready yet, no worries, uncomment the bash command bellow and run it so the data download and prepare to use Keras Data Preprocessor.

In [1]:
import os

from tensorflow import keras
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [2]:
# Download and prepare data (it may take a while, ~250mb)
#os.system("poetry run python data_preprocessing.py")

In [3]:
train = image_dataset_from_directory(
    directory='data/train/',
    labels='inferred',
    label_mode='categorical',
    batch_size=32,
    image_size=(256, 256)
)

Found 11490 files belonging to 8 classes.


In [4]:
val = image_dataset_from_directory(
    directory='data/val/',
    labels='inferred',
    label_mode='categorical',
    batch_size=32,
    image_size=(256, 256)
)

Found 1272 files belonging to 8 classes.


In [6]:
model = keras.applications.Xception( 
    weights=None,
    input_shape=(256, 256, 3), 
    classes=8,
)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit(train, epochs=1, validation_data=val)

  1/360 [..............................] - ETA: 5:53:19 - loss: 2.1711